In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

# Find the spark
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


In [ ]:
mydata = spark.read.format("csv").option("header", "true").load("/content/original.csv")

In [ ]:
mydata.show()

In [ ]:
# replace the null
from pyspark.sql.functions import *

# replace the null value in column City, and create new colum namned clean_city
mydata2 = mydata.withColumn('clean_city', when(mydata.City.isNull(), 'Unknown').otherwise(mydata.City))

In [ ]:
# filter value in colum is not null
mydata2 = mydata2.filter(mydata2.JobTitle.isNotNull())

In [ ]:
# remove first character
mydata2 = mydata2.withColumn('clean_salary', mydata2.Salary.substr(2,100).cast('float'))

In [ ]:
# calculate average clean_salary
mean = mydata2.groupBy().avg('clean_salary')
mean.show()

In [ ]:
# because mean is data frame -> take the fist row + first column
mean = mydata2.groupBy().avg('clean_salary').take(1)[0][0] # first row and first column
print(mean)

In [ ]:
from pyspark.sql.functions import lit

# replace null value with mean value
mydata2 = mydata2.withColumn('new_salary', when(mydata2.clean_salary.isNull(), mean).otherwise(mydata2.clean_salary))

In [ ]:
import numpy as np
# filter latitudes isNotNul()
latitudes = latitudes.filter(latitudes.Latitude.isNotNull())

In [ ]:
# convert string to float
latitudes = latitudes.withColumn('latitude2', latitudes.Latitude.cast('float')).select('latitude2')

In [ ]:
# calculate the median
median = np.median(latitudes.collect())
print(median)

In [ ]:
# replace the null values in Latitude with median

In [ ]:
men_above_average = mydata2.filter((mydata2.gender == "Male") & (mydata2.clean_salary > mean_salary)).count()
women_above_average = mydata2.filter((mydata2.gender == "Women") & (mydata2.clean_salary > mean_salary)).count()

# Print the results
if men_above_average > women_above_average:
  print("Men are more likely to earn more than the average salary.")
elif women_above_average > men_above_average:
  print("Women are more likely to earn more than the average salary.")
else:
  print("Men and women are equally likely to earn more than the average salary.")


In [ ]:
import pyspark.sql.functions as sqlfunc

genders = mydata2.groupBy('gender').agg(sqlfunc.avg('new_salary').alias('avg_salary'))
genders.show()

In [ ]:
df = mydata2.withColumn('female_salary', when(mydata2.gender == 'Female', mydata2.new_salary).otherwise(lit(0)))
df = df.withColumn('male_salary', when(mydata2.gender == 'Male', mydata2.new_salary).otherwise(lit(0)))
df.show()

In [ ]:
df = df.groupBy('JobTitle').agg(sqlfunc.avg('female_salary').alias('avg_female_salary'), sqlfunc.avg('male_salary').alias('avg_male_salary'))

In [ ]:
df = df.withColumn('delta', df.avg_female_salary - df.avg_male_salary)
df.show()

In [ ]:
# find down which cities has the highest average salary
cityavg = mydata2.groupBy('City').agg(sqlfunc.avg('new_salary').alias('final_avg_salary'))
cityavg.show()

In [ ]:
# sort the final_avg_salary form high to low
sorted_cityavg = cityavg.sort(desc('final_avg_salary'))
sorted_cityavg.show()